Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [37]:
# функция для обработки заработной платы
def string_salary(text):
    salary_min = None
    salary_max = None
    currency = None
    str_min = []
    str_max = []
    str_currency = []
    string_list = text.split()
    string = ''.join(string_list)
    if string[0] == 'о':
        for i in range(2, len(string)):
            if string[i].isdigit():
                str_min.append(string[i])
            else:
                str_currency.append(string[i])
        salary_min = float(''.join(str_min))
        currency = ''.join(str_currency)
    elif string[0] == 'д':
        for i in range(2, len(string)):
            if string[i].isdigit():
                str_max.append(string[i])
            else:
                str_currency.append(string[i])
        salary_max = float(''.join(str_max))
        currency = ''.join(str_currency)
    else:
        i = 0
        flag = 0
        while i < (len(string)):
            if flag == 0:
                if string[i].isdigit():
                    str_min.append(string[i])
                else:
                    flag = 1
            else:
                if string[i].isdigit():
                    str_max.append(string[i])
                else:
                    str_currency.append(string[i])
            i += 1
        salary_max = float(''.join(str_max))
        salary_min = float(''.join(str_min))
        currency = ''.join(str_currency)
    return salary_min, salary_max, currency

In [56]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import pandas as pd

data = pd.DataFrame(columns=['name', 'salary_min', 'salary_max', 'currency', 'source', 'link'])

vacancy = 'юрист'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
           'Accept':'*/*'}

main_link = 'https://www.hh.ru'

k = 0

while True:
    
    params = {
        'clusters': 'true',
        'enable_snippets': 'true',
        'st': 'searchVacancy',
        'text': vacancy,
        'fromSearch': 'true',
        'page': k
             }

    response = requests.get(f'{main_link}/search/vacancy', headers=headers, params=params)

    soup = bs(response.text, 'lxml')

    vacancy_block = soup.find('div',{'class':'vacancy-serp'})
    
    if vacancy_block is None:
        break

    vacancies_list = vacancy_block.findAll('div',{'class':'vacancy-serp-item'})


    for i in vacancies_list:
        name_link_block = i.find('a',{'class':'bloko-link HH-LinkModifier'})
        link = name_link_block['href']
        name = name_link_block.text
        salary_block = i.find('div',{'class':'vacancy-serp-item__sidebar'})
        salary_min = None
        salary_max = None
        currency = None
        if salary_block.text != '':
            salary_min, salary_max, currency = string_salary(salary_block.text)
        data = data.append(
        {
            "name": name,
            "link": link,
            "salary_min": salary_min,
            "salary_max": salary_max,
            "currency": currency,
            "source": 'hh.ru',
        },
        ignore_index=True)
    k += 1

data

,name,salary_min,salary_max,currency,source,link
0,Ведущий юрист,35000,70000,руб.,hh.ru,https://hh.ru/vacancy/37384273?query=%D1%8E%D1...
1,Юрист,80000,200000,руб.,hh.ru,https://hh.ru/vacancy/37374293?query=%D1%8E%D1...
2,Помощник юриста,1000,1000,USD,hh.ru,https://hh.ru/vacancy/36320970?query=%D1%8E%D1...
3,Юрисконсульт,700,1000,бел.руб.,hh.ru,https://hh.ru/vacancy/37074171?query=%D1%8E%D1...
4,Юрист,8e+06,None,сум,hh.ru,https://hh.ru/vacancy/37058272?query=%D1%8E%D1...
...,...,...,...,...,...,...
1995,Юрист,None,None,None,hh.ru,https://hh.ru/vacancy/37158095?query=%D1%8E%D1...
1996,Юрисконсульт,50000,None,руб.,hh.ru,https://hh.ru/vacancy/37154045?query=%D1%8E%D1...
1997,Юрист по корпоративному праву,None,60000,руб.,hh.ru,https://hh.ru/vacancy/36122868?query=%D1%8E%D1...
1998,Юрист в единственном лице в Таганрог,None,None,None,hh.ru,https://hh.ru/vacancy/37243982?query=%D1%8E%D1...
